<a href="https://colab.research.google.com/github/bin-crypto-test/BERTopic/blob/master/BERTopic_(semi)_supervised_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial** - (semi)-supervised topic modeling
(last updated 26-04-2021)

In this tutorial, we will be looking at a new feature of BERTopic, namely (semi)-supervised topic modeling! This allows us to steer the dimensionality reduction of the embeddings into a space that closely follows any labels you might already have. 

## Semi-supervised modeling
(semi)-supervised topic modeling is a class of methods that allows the user to perform topic modeling with previously defined labels. This might help nudge the model towards specific topics or classes for which you have labels. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts that each is assigned to one of 20 topics:

In [ ]:
import pandas as pd
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))
docs = data["data"]
targets = data["target"]
target_names = data["target_names"]
classes = [data["target_names"][i] for i in data["target"]]

Each document can be put into one of the following categories:

In [ ]:
target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

# **(semi)-Supervised modeling**


## Basic Model
Before we start with semi-supervised modeling, let us first take a look at the output of the basic model.

In [ ]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs)

2021-04-25 05:59:40,718 - BERTopic - Transformed documents to Embeddings


2021-04-25 05:59:56,400 - BERTopic - Reduced dimensionality with UMAP
2021-04-25 05:59:57,376 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name
0,-1,8820,-1_me_god_my_do
1,24,864,24_game_hockey_nhl_10
2,4,774,4_nasa_orbit_spacecraft_moon
3,47,519,47_food_medical_diet_vitamin
4,31,500,31_hit_pitching_baseball_pitcher
5,21,470,21_key_encryption_keys_nsa
6,2,446,2_zzzzzzt_ergonomics_ergonomicallyhellish_ergo...
7,26,357,26_wiring_amp_wires_audio
8,58,332,58_bike_bikes_motorcycle_brake
9,93,213,93_homosexual_homosexuality_sex_homosexuals


The topics that were created mostly make sense. There are some clearly defined topics such as "nasa, orbit, spacecraft, moon" but also some topics that seem mostly derived from other topics. We can visualize this by extracting the topic representations per class and see if our unsupervised model closely resembles this. 

**NOTE**: You can **hover** over the bars to see the representation per class!!

In [ ]:
topics_per_class = topic_model.topics_per_class(docs, topics, classes=classes)
fig_unsupervised = topic_model.visualize_topics_per_class(topics_per_class, top_n=10)
fig_unsupervised

20it [00:36,  1.84s/it]


The results do seem promising. Topics like "nasa, space, etc" seem to be clearly related to sci.space, but some topics were created that span many categories. For example, we expect the topic "bike, bikes, etc"  to only appear in rec.motorcycles.  

## Semi-supervised
In the example above you might notice that some topics were somewhat smushed together. What we would like to see is a clear separation between those topics. Fortunately, we have to labels and can use them to improve the model. 

Since we are not interested in any other topics, this method is called semi-supervised topic modeling. In practice, this means that we have the labels of some documents but not all. 

For this example let's say we only have the labels of all computer-related categories:

In [ ]:
 labels_to_add = ['comp.graphics', 'comp.os.ms-windows.misc',
                  'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
                  'comp.windows.x',]
 indices = [target_names.index(label) for label in labels_to_add]
 new_labels = [label if label in indices else -1 for label in targets]

When generating our new labels it is important to mark unknown classes as **-1**. Next, we use those newly constructed labels to again run BERTopic:

In [ ]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs, y=new_labels)

2021-04-25 05:55:43,262 - BERTopic - Transformed documents to Embeddings


2021-04-25 05:56:03,260 - BERTopic - Reduced dimensionality with UMAP
2021-04-25 05:56:05,695 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name
0,-1,7167,-1_my_me_all_no
1,24,873,24_game_hockey_550_nhl
2,31,804,31_window_server_widget_openwindows
3,9,759,9_space_nasa_orbit_spacecraft
4,52,739,52_jpeg_image_images_file
5,26,738,26_baseball_games_hit_pitching
6,20,613,20_food_medical_vitamin_diet
7,115,572,115_god_jesus_christ_church
8,0,517,0_zzzzzzt_ergo_ergonomicallyhellish_ergonomica...
9,48,506,48_key_encryption_keys_nsa


Finally, we can again extract the topics per class to see if our semi-supervised approach had some effect:

In [ ]:
topics_per_class = topic_model.topics_per_class(docs, topics, classes=classes)
fig_semi_supervised = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10)
fig_semi_supervised

20it [00:28,  1.45s/it]


We can clearly see that many more topics about computers were created and that the seperation between those topics are solid. This indicates that even if you do not have all the labels, you can definitely improve the model!

However, there are still some clusters that could be improved with the labels that we have. 

## Supervised

Finally, we are going to be using all labels. These labels help BERTopic understand where most clusters can be found. However, this does not mean that it will only find the 20 clusters that we have defined. If there are sub-clusters to be found, then there is a good chance BERTopic will find them! 

In [ ]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs, y=targets)

2021-04-19 14:08:06,932 - BERTopic - Transformed documents to Embeddings


2021-04-19 14:08:24,945 - BERTopic - Reduced dimensionality with UMAP
2021-04-19 14:08:25,735 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name
0,-1,2917,-1_who_data_no_me
1,5,966,5_hockey_25_550_nhl
2,41,951,41_church_jesus_christ_christians
3,30,929,30_drive_controller_drives_bios
4,6,926,6_sale_price_100_sell
5,64,902,64_window_server_widget_display
6,2,878,2_game_baseball_games_last
7,4,841,4_space_nasa_orbit_spacecraft
8,57,832,57_gun_guns_firearms_fbi
9,13,821,13_mac_apple_monitor_disk


Not only do we see a nice seperation of the topics, there are significantly less outliers which shows that BERTopic has improved in connecting the documents to topics. 

Let's see the results by again visualizing the topic representation per class:

In [ ]:
topics_per_class = topic_model.topics_per_class(docs, topics, classes=classes)

20it [00:28,  1.42s/it]


In [ ]:
fig_supervised = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10)
fig_supervised

Now that we have used all labels, BERTopic seems to closely match our pre-defined labels. Moreover, it still allows to discover topics that were not previously defined. Thus, you can use this method to find unknown topics in pre-defined topics!